# Expansion Approach 
The change in intensity between two points in time with slightly different concentration hinges on a difference of exponential terms. This term can be manipulated to obtain a $\Delta c = c_1 - c_2$ (with c1 and c2 being the concentrations at two points). This tells us how much the intensity values should change for any given $\Delta c$. If this change is smaller than our noise margin, we would not be able to detect it. This then sets a lower bound on our sensitivity. 

## Equations
Let's assume we receive N photons in total. All the terms in the intensity equation we do not care about make up $G_i$. The path for each photon through our layer of interest is $L_i$. Then, the difference in intensity between two points (assuming all else stays same)
$$
I_1 - I_2 = \Delta I = \sum G_i (e^{-\epsilon c_1 L_i} - e^{-\epsilon c_2 L_i})
$$
The exponential difference term here can be expanded as,
$$
e^{-\epsilon c_1 L_i} - e^{-\epsilon c_2 L_i} = 1 -\epsilon c_1 L_i + \frac{(\epsilon c_1 L_i)^2}{2} - ... - 1 + \epsilon c_2 L_i - \frac{(\epsilon c_2 L_i)^2}{2} + ... = -\epsilon\Delta c L_i + \frac{(\epsilon L_i)^2\Delta c \bar{c}}{2} - ...
$$

Replacing this back into the original eqn,
$$
\Delta I = -\epsilon\Delta c \sum G_i L_i + \frac{\epsilon^2\Delta c \bar{c}}{2} \sum G_i (L_i)^2 - = -\epsilon\Delta c <G,L> + \frac{\epsilon^2\Delta c \bar{c}}{2} <G, L^2> - 
$$

How small are these terms and at what point can we ignore them?

## Notes
1. $L_i$ is the partial path through our layer of interest. This is usually going to be a large term. Specifically for the far away detectors. When squared, the results would become even larger! >.<

## Testing The Error In Approximations

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from inverse_modelling_tfo.tools.name_decoder import decode_extended_filename

# NOTE: All the intensity values consider every detector in the concentric cirlces. Which means they
# need to be dividided by the individual detector count at some point. But for these comparisons, 
# all sides get the same multiplier -> don't really need to concern about it as now
# Pre-calculated intensity using the regular formula (inner product)
INTENSITY_DATA_PATH = Path('../data/s_based_intensity_low_conc2.pkl')

intensity_data = pd.read_pickle(INTENSITY_DATA_PATH)
RAW_SIM_DATA_PATH = '/home/rraiyan/simulations/tfo_sim/data/raw_dan_iccps_equispace_detector/fa_1_wv_1_sa_0.1_ns_1_ms_2_ut_5.pkl'
raw_sim_data = pd.read_pickle(RAW_SIM_DATA_PATH)
maternal_wall_thickness, uterus_thickness, wave_int = decode_extended_filename(RAW_SIM_DATA_PATH)
intensity_data = intensity_data[(intensity_data['Maternal Wall Thickness'] == maternal_wall_thickness) & (intensity_data['Wave Int'] == wave_int)]

In [7]:
intensity_data.groupby('Fetal Hb Concentration').groups.keys()

dict_keys([0.11, 0.125, 0.14, 0.15500000000000003, 0.17])

In [8]:
MATERNAL_Hb = 12.
MATERNAL_SAT = 0.9
FETAL_SAT = 0.225
FETAL_Hb1 = 0.11
FETAL_Hb2 = 0.14

i1 = intensity_data[(intensity_data["Maternal Hb Concentration"] == MATERNAL_Hb) & (intensity_data["Maternal Saturation"] == MATERNAL_SAT) & (intensity_data["Fetal Saturation"] == FETAL_SAT) & (intensity_data["Fetal Hb Concentration"] == FETAL_Hb1)]
i2 = intensity_data[(intensity_data["Maternal Hb Concentration"] == MATERNAL_Hb) & (intensity_data["Maternal Saturation"] == MATERNAL_SAT) & (intensity_data["Fetal Saturation"] == FETAL_SAT) & (intensity_data["Fetal Hb Concentration"] == FETAL_Hb2)]
intensity_difference = i1['Intensity'].to_numpy() - i2['Intensity'].to_numpy()  # NON-LOG Difference

In [9]:
from inverse_modelling_tfo.tools.s_based_intensity_datagen import MU_MAP_BASE1, MU_MAP_BASE2, get_mu_a
# Create SDD column!
raw_sim_data['SDD'] = raw_sim_data['X'] - 100

In [18]:
# Mu Map
modified_mu_map = MU_MAP_BASE1.copy() if wave_int == 1 else MU_MAP_BASE2
modified_mu_map[1] = get_mu_a(MATERNAL_SAT, MATERNAL_Hb, wave_int)
modified_mu_map['c1'] = get_mu_a(FETAL_SAT, FETAL_Hb1, wave_int)
modified_mu_map['c2'] = get_mu_a(FETAL_SAT, FETAL_Hb2, wave_int)
all_G = []
all_aprrox_1term = []
all_aprrox_2term = []
all_L4 = []

delta_c = FETAL_Hb1 - FETAL_Hb2
c_bar = (FETAL_Hb1 + FETAL_Hb2)/2
epsilon = get_mu_a(FETAL_SAT, FETAL_Hb1, wave_int) / FETAL_Hb1  # mu_a = epsilon * c

for sdd in i1["SDD"].to_numpy():
    filtered_photon_data = raw_sim_data[raw_sim_data['SDD'] == sdd]
    G = filtered_photon_data[['L1 ppath', 'L2 ppath', 'L3 ppath']].to_numpy()
    for i in range(1, 4):
        G[:, i - 1] = np.exp(-modified_mu_map[i] * G[:, i - 1])
    G = np.prod(G, axis=1)
    L4 = filtered_photon_data['L4 ppath'].to_numpy()
    approx1 = - delta_c * epsilon * np.dot(G, L4)
    approx2 = approx1 + epsilon**2 * delta_c * c_bar * np.dot(G, np.square(L4))
    
    all_G.append(G)
    all_L4.append(L4)
    all_aprrox_1term.append(approx1)
    all_aprrox_2term.append(approx2)

all_aprrox_1term = np.array(all_aprrox_1term)
all_aprrox_2term = np.array(all_aprrox_2term)

In [20]:
intensity_diff_error_df = pd.DataFrame({
    'SDD': i1['SDD'].to_numpy(),
    'Pre-calculated' : intensity_difference,
    'Approx 1 Term': all_aprrox_1term,
    'Approx 2 Terms' : all_aprrox_2term,
    'Approx 1 %Error': np.abs(all_aprrox_1term - intensity_difference)/intensity_difference * 100,
    'Approx 2 %Error' : np.abs(all_aprrox_2term - intensity_difference)/intensity_difference * 100,
})
print("Aboslute Error")
display(intensity_diff_error_df)

Aboslute Error


,SDD,Pre-calculated,Approx 1 Term,Approx 2 Terms,Approx 1 %Error,Approx 2 %Error
0,10,7067.201298,24077.464798,117825.588305,240.693066,1.567217e+03
1,14,6396.884950,27517.150217,153024.327491,330.164845,2.292169e+03
2,19,4468.396643,26818.770749,174119.378457,500.187783,3.796686e+03
3,23,2945.693413,25175.459621,188298.832815,754.653085,6.292343e+03
4,28,1864.231994,22871.489566,190736.364608,1126.858548,1.013136e+04
5,32,1157.426190,20566.779123,190967.463805,1676.940880,1.639932e+04
6,37,683.372634,18007.670310,184435.873815,2535.117273,2.688906e+04
7,41,417.049346,15823.257842,175831.954657,3694.097264,4.206095e+04
8,46,242.866019,13586.946743,163571.927702,5494.420651,6.725068e+04
9,50,146.449600,11800.288381,151541.133453,7957.576361,1.033766e+05


__Remark__ : Well, the approximations are way off. Especially for the far detectors. Which makes sense. The $L_i$ for those guys is going to be much larger. Meaning taking their higher powers would only make them bigger. Also, weird thing, adding the second term seems to increase the error!  
(Check this bottom cell out)

In [29]:
from math import e

# Think of all the constant terms (epsilon and L) as x and the c terms as a or b
# Large a and b values
a, b, x = 6, 4, 1
print('True Value : ', e**(-a * x) - e**(-b * x))
approx = x * (b - a) + 1/2 * x**2 * (a**2 - b**2) + 1/6 * x**3 * (b*3 - a*3) + 1/24 * x**4 * (a**4 - b**4) + 1/120 *  x**5 * (b**5 - a**5) + 1/720 * x**6 * (a**6 - b**6)   # Upto 6th order terms
print('Approx Value(6th order) : ', approx)

# Fractional a and b
a, b, x = 0.6, 0.5, 1
print('True Value : ', e**(-a * x) - e**(-b * x))
approx = x * (b - a) + 1/2 * x**2 * (a**2 - b**2) + 1/6 * x**3 * (b*3 - a*3) + 1/24 * x**4 * (a**4 - b**4) + 1/120 *  x**5 * (b**5 - a**5) + 1/720 * x**6 * (a**6 - b**6)   # Upto 6th order terms
print('Approx Value(6th order) : ', approx)




True Value :  -0.015836886712067826
Approx Value(6th order) :  53.17777777777778
True Value :  -0.057719023618606924
Approx Value(6th order) :  -0.09254865138888883


For fractional a and b, the approximation actually works pretty well but for large values of a and b, the approximation fails. (It's not the difference between a and b but rather the values themselves need to be smaller than 1)